<a href="https://colab.research.google.com/github/Kussil/CVX_Rice_project/blob/main/03_Sentiment_Analysis/Sentiment_Analysis_(FinBERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add Sentiment Analysis

In [1]:
#installs

In [2]:
#imports
import os
from google.colab import userdata
import pandas as pd
import math
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch

### Import Files From GitHub

In [3]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/CVX_Rice_project.git

Cloning into 'CVX_Rice_project'...
remote: Enumerating objects: 1412, done.
remote: Counting objects: 100% (401/401), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 1412 (delta 218), reused 389 (delta 210), pack-reused 1011
Receiving objects: 100% (1412/1412), 165.34 MiB | 9.46 MiB/s, done.
Resolving deltas: 100% (809/809), done.
Updating files: 100% (1064/1064), done.


In [4]:
invest_df = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/Investment_Research.csv')
proquest_df = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/ProQuest_Articles.csv')
# sec_df = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/SEC_Filings.csv')

# df = pd.concat([invest_df, proquest_df, sec_df], ignore_index=True)
df = pd.concat([invest_df, proquest_df], ignore_index=True)
df['Article Index'] = df.index
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df['Date']= pd.to_datetime(df['Date']).dt.date
df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4
...,...,...,...,...,...,...,...,...
8599,ProQuest,2.238654e+09,PSX,2019-06-12,"As the Price of Gas Tumbles, Even California I...",Gasoline prices have been falling fast in rece...,https://www.proquest.com/newspapers/as-price-g...,8599
8600,ProQuest,2.239478e+09,PSX,2019-06-10,"Phillips 66, Plains plan pipeline system",CUSHING Phillips 66 and Plains All American Pi...,https://www.proquest.com/newspapers/phillips-6...,8600
8601,ProQuest,2.238860e+09,PSX,2019-06-07,Autonomous robot inspects in-service diesel st...,"HOUSTON Phillips 66, Square Robot Inc. and Ver...",https://www.proquest.com/newspapers/autonomous...,8601
8602,ProQuest,2.233474e+09,PSX,2019-06-03,United States/Mexico: Report: Trump's Tariff o...,US President Donald Trump's threats to tax Mex...,https://www.proquest.com/newspapers/united-sta...,8602


### Chunk Text for FinBERT, < 512 token input

In [5]:
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

token_limit = 500

def chunk_contents(x):
  x_str = str(x)
  tokens = tokenizer(x_str)['input_ids']
  token_count = len(tokens)
  chunk_count = math.ceil(token_count / token_limit)
  chunk_list = []
  for i in range(chunk_count):
    chunk_list.append(tokenizer.decode(tokens[i*token_limit:(i+1)*token_limit]))
  return chunk_list

df['Text Chunks'] = df['Article Text'].apply(chunk_contents)
df

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1816 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

### Apply FinBERT Sentiment Analysis
Create Neurtal, Positive, Negative matrix for each chunk

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

sentiment_checkpoint = 'ProsusAI/finbert'
softmax = torch.nn.Softmax(dim=1)

######## Sentiment Analysis ############################
finbert = BertForSequenceClassification.from_pretrained(sentiment_checkpoint)#,num_labels=3)
finbert.to(device)
tokenizer = BertTokenizer.from_pretrained(sentiment_checkpoint)

labels = {0:'neutral', 1:'positive',2:'negative'}

def finbert_sentiment(chunk_list):
  with torch.no_grad():
    content_sentiment_list = []
    for chunk in chunk_list:
      inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)
      inputs.to(device)
      outputs = finbert(**inputs)[0]
      softmax_outputs = softmax(outputs)[0]
      content_sentiment_list.append([softmax_outputs[0].item(), softmax_outputs[1].item(), softmax_outputs[2].item()])
  return content_sentiment_list

df['Chunk Sentiments'] = df['Text Chunks'].apply(finbert_sentiment)
df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks,Chunk Sentiments
0,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,"Stock Report | March 12, 2022 | NYSE Symbol: C...",NaN,0,"[[CLS] stock report | march 12, 2022 | nyse sy...","[[0.05401257053017616, 0.01840130239725113, 0...."
1,Investment Research,NaN,CVX,2024-05-13,CFRA LIFTS VIEW ON SHARES OF CHEVRON CORPORATI...,"Stock Report | August 05, 2023 | NYSE Symbol: ...",NaN,1,"[[CLS] stock report | august 05, 2023 | nyse s...","[[0.05142822861671448, 0.018504083156585693, 0..."
2,Investment Research,NaN,CVX,2024-05-11,Chevron Corporation,"Stock Report | March 04, 2023 | NYSE Symbol: C...",NaN,2,"[[CLS] stock report | march 04, 2023 | nyse sy...","[[0.051014646887779236, 0.018687281757593155, ..."
3,Investment Research,NaN,CVX,2024-05-04,Chevron Corporation,"Stock Report | October 17, 2020 | NYSE Symbol:...",NaN,3,"[[CLS] stock report | october 17, 2020 | nyse ...","[[0.04866460710763931, 0.018545862287282944, 0..."
4,Investment Research,NaN,CVX,2024-05-01,Chevron Corporation,"Stock Report | December 25, 2021 | NYSE Symbol...",NaN,4,"[[CLS] stock report | december 25, 2021 | nyse...","[[0.04953540116548538, 0.019073335453867912, 0..."
...,...,...,...,...,...,...,...,...,...,...
4517,ProQuest,2.464845e+09,XOM,2020-11-28,Top Pension Sold Bank America Wells Fargo Exxon,The State of Wisconsin Investment Board made s...,https://www.proquest.com/newspapers/top-pensio...,4517,[[CLS] the state of wisconsin investment board...,"[[0.04513585940003395, 0.8997517824172974, 0.0..."
4518,ProQuest,2.464567e+09,XOM,2020-11-28,Exxons Bass Asset Sale Hits Stormy Waters,ExxonMobil has called off the potential multib...,https://www.proquest.com/newspapers/exxons-bas...,4518,[[CLS] exxonmobil has called off the potential...,"[[0.18219533562660217, 0.5766971111297607, 0.2..."
4519,ProQuest,2.464604e+09,XOM,2020-11-27,Why Exxons Cash Flow Still Wont Cover Whole,A recent rise in oil prices isn't enough to fu...,https://www.proquest.com/newspapers/why-exxons...,4519,[[CLS] a recent rise in oil prices isn't enoug...,"[[0.229207381606102, 0.7171589136123657, 0.053..."
4520,ProQuest,2.464542e+09,XOM,2020-11-27,Exxon Internally Shows More Pessimism On Oil,Exxon Mobil Corp. has lowered its outlook on o...,https://www.proquest.com/newspapers/exxon-inte...,4520,[[CLS] exxon mobil corp. has lowered its outlo...,"[[0.013398897834122181, 0.9437286853790283, 0...."


Merge chunk sentiments into text sentiment

In [ ]:
def average_sentiments(chunk_sentiment_list):
  sentiment_sum = [sum(x) for x in zip(*chunk_sentiment_list)]
  sentiment_avg = [x / len(chunk_sentiment_list) for x in sentiment_sum]
  return sentiment_avg

# Summarize Chunk Sentiments into an Article Sentiment
df['Sentiments']= df['Chunk Sentiments'].apply(average_sentiments)
# Split the 'Sentiments' column into three separate columns
df[['Neutral', 'Positive', 'Negative']] = pd.DataFrame(df['Sentiments'].tolist(), index=df.index)
df = df.drop('Sentiments', axis=1)
df = df.drop('Chunk Sentiments', axis=1)
df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Sentiments,Neutral,Positive,Negative
0,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,"Stock Report | March 12, 2022 | NYSE Symbol: C...",NaN,0,"[0.46704853093251586, 0.05334289465099573, 0.4...",0.467049,0.053343,0.479609
1,Investment Research,NaN,CVX,2024-05-13,CFRA LIFTS VIEW ON SHARES OF CHEVRON CORPORATI...,"Stock Report | August 05, 2023 | NYSE Symbol: ...",NaN,1,"[0.17823628336191177, 0.1641262248158455, 0.65...",0.178236,0.164126,0.657638
2,Investment Research,NaN,CVX,2024-05-11,Chevron Corporation,"Stock Report | March 04, 2023 | NYSE Symbol: C...",NaN,2,"[0.17845207313075662, 0.20219592656940222, 0.6...",0.178452,0.202196,0.619352
3,Investment Research,NaN,CVX,2024-05-04,Chevron Corporation,"Stock Report | October 17, 2020 | NYSE Symbol:...",NaN,3,"[0.10703014209866524, 0.2638334264047444, 0.62...",0.107030,0.263833,0.629136
4,Investment Research,NaN,CVX,2024-05-01,Chevron Corporation,"Stock Report | December 25, 2021 | NYSE Symbol...",NaN,4,"[0.3376919333823025, 0.03759842738509178, 0.62...",0.337692,0.037598,0.624710
...,...,...,...,...,...,...,...,...,...,...,...,...
4517,ProQuest,2.464845e+09,XOM,2020-11-28,Top Pension Sold Bank America Wells Fargo Exxon,The State of Wisconsin Investment Board made s...,https://www.proquest.com/newspapers/top-pensio...,4517,"[0.027041552122682333, 0.9343084096908569, 0.0...",0.027042,0.934308,0.038650
4518,ProQuest,2.464567e+09,XOM,2020-11-28,Exxons Bass Asset Sale Hits Stormy Waters,ExxonMobil has called off the potential multib...,https://www.proquest.com/newspapers/exxons-bas...,4518,"[0.2657018005847931, 0.4268864840269089, 0.307...",0.265702,0.426886,0.307412
4519,ProQuest,2.464604e+09,XOM,2020-11-27,Why Exxons Cash Flow Still Wont Cover Whole,A recent rise in oil prices isn't enough to fu...,https://www.proquest.com/newspapers/why-exxons...,4519,"[0.11878884956240654, 0.8157167434692383, 0.06...",0.118789,0.815717,0.065494
4520,ProQuest,2.464542e+09,XOM,2020-11-27,Exxon Internally Shows More Pessimism On Oil,Exxon Mobil Corp. has lowered its outlook on o...,https://www.proquest.com/newspapers/exxon-inte...,4520,"[0.011563639777402083, 0.9566905299822489, 0.0...",0.011564,0.956691,0.031746


In [ ]:
#create chunk dataframe
df_chunk = df.explode('Text Chunks').reset_index(drop=True)
df_chunk = df_chunk.drop('Article Text', axis=1)
df_chunk['Ticker'] = df_chunk['Ticker'].fillna('')
#df_chunk = df_chunk.drop('Chunk Sentiments', axis=1)

# remove chunks from main dataframe
df = df.drop('Text Chunks', axis=1)

df_chunk

,Source,Unique_ID,Ticker,Date,Article Headline,URL,Article Index,Neutral,Positive,Negative,Text Chunks
0,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,NaN,0,0.467049,0.053343,0.479609,"[CLS] stock report | march 12, 2022 | nyse sym..."
1,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,NaN,0,0.467049,0.053343,0.479609,", 865 2018 35, 968 40, 491 42, 105 40, 338 158..."
2,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,NaN,0,0.467049,0.053343,0.479609,capital to dividends and buybacks. urisks to o...
3,Investment Research,NaN,CVX,2024-05-13,Chevron Corporation,NaN,0,0.467049,0.053343,0.479609,specific person who may receive this report. i...
4,Investment Research,NaN,CVX,2024-05-13,CFRA LIFTS VIEW ON SHARES OF CHEVRON CORPORATI...,NaN,1,0.178236,0.164126,0.657638,"[CLS] stock report | august 05, 2023 | nyse sy..."
...,...,...,...,...,...,...,...,...,...,...,...
14420,ProQuest,2.464604e+09,XOM,2020-11-27,Why Exxons Cash Flow Still Wont Cover Whole,https://www.proquest.com/newspapers/why-exxons...,4519,0.118789,0.815717,0.065494,% from its close on wednesday. exxon shares ar...
14421,ProQuest,2.464542e+09,XOM,2020-11-27,Exxon Internally Shows More Pessimism On Oil,https://www.proquest.com/newspapers/exxon-inte...,4520,0.011564,0.956691,0.031746,[CLS] exxon mobil corp. has lowered its outloo...
14422,ProQuest,2.464542e+09,XOM,2020-11-27,Exxon Internally Shows More Pessimism On Oil,https://www.proquest.com/newspapers/exxon-inte...,4520,0.011564,0.956691,0.031746,##investment in oil and gas production would l...
14423,ProQuest,2.464542e+09,XOM,2020-11-27,Exxon Internally Shows More Pessimism On Oil,https://www.proquest.com/newspapers/exxon-inte...,4520,0.011564,0.956691,0.031746,years argued that the company is extremely con...


Write to GitHub Repo

In [ ]:
# save csv locally
df.to_csv('/content/sample_data/sentiment_data.csv', index=True)
df_chunk.to_csv('/content/sample_data/sentiment_chunkdata.csv', index=True)

In [ ]:
#Copy CSV to GitHub Repository
!cp /content/sample_data/sentiment_data.csv /content/CVX_Rice_project/03_Sentiment_Analysis/
!cp /content/sample_data/sentiment_chunkdata.csv /content/CVX_Rice_project/03_Sentiment_Analysis/

In [ ]:
#Commit and Push Changes
!git config --global user.email "bw58@rice.edu"
%cd /content/CVX_Rice_project/03_Sentiment_Analysis/
!git add sentiment_data.csv
!git commit -m "Add sentiment_data.csv"
!git add sentiment_chunkdata.csv
!git commit -m "Add sentiment_chunkdata.csv"
!git push origin main

/content/CVX_Rice_project/03_Sentiment_Analysis
[main 0047845] Add sentiment_data.csv
 1 file changed, 254073 insertions(+)
 create mode 100644 03_Sentiment_Analysis/sentiment_data.csv
[main 9985215] Add sentiment_chunkdata.csv
 1 file changed, 14426 insertions(+)
 create mode 100644 03_Sentiment_Analysis/sentiment_chunkdata.csv
Enumerating objects: 10, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 9.41 MiB | 1.96 MiB/s, done.
Total 8 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 1 local object.
To https://github.com/Kussil/CVX_Rice_project.git
   8d79566..9985215  main -> main


In [ ]:
#!rm -rf /content/CVX_Rice_project/